<a href="https://colab.research.google.com/github/KrzysztofLin/WRC_car_dashboard/blob/main/WRC_car_dashboard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install dash
!pip install jupyter_dash
!pip install dash_bootstrap_components

In [ ]:
import cv2
import numpy as np
import pandas as pd
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt
import plotly.graph_objects as go

from tqdm import trange

from dash.dependencies import Input, Output
from jupyter_dash import JupyterDash

import dash
import dash_core_components as dcc
import dash_html_components as html
import dash_bootstrap_components as dbc

import plotly.figure_factory as ff
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Color and dashbord's object positions adjustments.

In [ ]:
BG_COLOR = '#4d4846'
FONT_COLOR = '#E4D6A7'
BG_PIC_COLOR = '#5e5551'
PAPER_BGCOLOR = '#6e6460'

SIDEBAR_STYLE = {
    'position': 'fixed',
    'top': 0,
    'left': 0,
    'bottom': 0,
    'width': '28rem',
    'padding': '2rem 1rem',
    'background-color': BG_COLOR,
    'color': FONT_COLOR
}

CONTENT_STYLE = {
    'margin-left': '28rem',
    'margin-right': '0rem',
    'padding': '2rem 1rem',
    'background-color': BG_COLOR,
    'color': FONT_COLOR,
    'display':'inline-block'
}

In [ ]:
from collections import defaultdict
def plot_3D_scatterplot(df):
  px.scatter_3d(df, y="G-Y", x="G-X", z="G-Z", color="G-Y", title="Wykres przeciążeń panujących w samochodzie")

def plot_graph_with_smooth_speed(dfa):
  n = 10 # the number of rows
  dfa = dfa[:-1]
  smooth_speed = dfa['SPEED'].groupby(np.arange(len(dfa['SPEED'])) // n).mean()
  smooth_df = pd.DataFrame()
  smooth_df['SPEED'] = smooth_speed
  smooth_df['UTC TIME'] = dfa['UTC TIME'].unique()
  fig = px.line(smooth_df, y='SPEED', x='UTC TIME')
  fig.update_layout(title_x=0.5, plot_bgcolor=BG_PIC_COLOR, paper_bgcolor=PAPER_BGCOLOR, xaxis_title="histogram")
  return fig


def plot_speed_vs_time_graph(dfa):
  indexer = pd.api.indexers.FixedForwardWindowIndexer(window_size=10)
  c = dfa['SPEED'].rolling(indexer).mean()
  c.iloc[:]
  fig = px.line(dfa, y=dfa['SPEED'], x=dfa['UTC TIME'], title="Wykres prędkości do czasu", color_discrete_sequence=['#fcd69a'])
  fig.update_layout(font_color='#E4D6A7', title_x=0.5, plot_bgcolor=BG_PIC_COLOR, paper_bgcolor=PAPER_BGCOLOR, xaxis_title="Godzina", yaxis_title = 'Prędkość pojazdu')
  fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='#807875')
  fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='#807875')

  return fig

def plot_histogram(dfa):
  
  # Create distplot with custom bin_size
  fig = px.histogram(dfa, x=['G-X', 'G-Z', 'G-Y'],color = 'variable', color_discrete_sequence = ['#fcd69a','#f5a364','#e0654c'])
  fig.update_layout(font_color='#E4D6A7', title_x=0.5, plot_bgcolor=BG_PIC_COLOR, paper_bgcolor=PAPER_BGCOLOR, xaxis_title="Wartość siły", yaxis_title = "Czestość")
  fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='#807875')
  fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='#807875')
  return fig

def calculate_mean_max_speed(dfa):
  
  print(f"AVG speed on rally stage {np.mean(dfa['SPEED']):.3f} km/h")
  print(f"Maximum speed on rally stage {np.max(dfa['SPEED']):.3f} km/h")

# Dodac kolorowanie przyspieszenia i hamowania jako color_discrete_sequence i color_discrete_map
# px.line(df_copy, 'LONGITUDE', 'LATITUDE', hover_data=["G-X", "G-Y", "G-Z", "SPEED"])

def plot_acceleration_graph(dfa):
  fig = go.Figure()
  fig.add_trace(go.Scatter(
     x=dfa['G-X'],
     y=dfa['G-Y'],
     mode='markers',
   
  ))  
  fig.update_xaxes(range=[-3, 1], zeroline=False)
  fig.update_yaxes(range=[-4, 4])

  px.scatter(x=dfa['G-X'], y=dfa['G-Y'])
  fig.update_traces(marker=dict(color="#f7c472"))
  fig.add_shape(type="circle",
      xref="x", yref="y",
      x0=-1.5, y0=-1, x1=-0.5, y1=1,
      line_color='#fcd69a',
  )
  fig.add_shape(type="circle",
      xref="x", yref="y",
      x0=-2, y0=-2, x1=0, y1=2,
      line_color='#f5a364',
  )
  fig.add_shape(type="circle",
      xref="x", yref="y",
      x0=-2.5, y0=-3, x1=0.5, y1=3,
      line_color='#e0654c',
  )
  fig.update_layout(title_text='Wykres przeciążeń', title_x=0.5, font_color='#E4D6A7', plot_bgcolor=BG_PIC_COLOR, paper_bgcolor=PAPER_BGCOLOR, xaxis_title="Wartość siły G-X", yaxis_title = "Wartość siły G-Y")
  fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='#807875')
  fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='#807875')
  return fig


In [ ]:
def show_graph_with_moving_dot(dfa):
  xy_array = dfa[['LONGITUDE', 'LATITUDE']].values
  xm = np.min(xy_array[:, 0])
  xM = np.max(xy_array[:, 0])
  ym = np.min(xy_array[:, 1])
  yM = np.max(xy_array[:, 1])

  zx = xy_array.copy()
  colors = ['aliceblue',  'aqua', 'aquamarine', 'darkturquoise']

  N = len(xy_array)
  fig = go.Figure(
      data=[go.Scatter(x=xy_array[:, 0], y=xy_array[:, 1], mode="lines", line=dict(width=2, color="#f7c472")),
            go.Scatter(x=xy_array[:, 0], y=xy_array[:, 1], mode="lines", line=dict(width=2, color="#f7c472")),],

      layout=go.Layout(
          xaxis=dict(range=[xm, xM], autorange=False, zeroline=False),
          yaxis=dict(range=[ym, yM], autorange=False, zeroline=False),
          title_font_color="#E4D6A7",
          legend_title_font_color="#E4D6A7",
          title_text="Trasa przejazdu", hovermode="closest",
          updatemenus=[dict(type="buttons",
                            buttons=[dict(label="Play",
                                          method="animate",
                                          args=[None, {"frame": {"duration": 5}}])])]),
      
      frames=[go.Frame(
          data=[go.Scatter(
              x=[zx[:, 0][k]],
              y=[zx[:, 1][k]],
              mode="markers",
              marker=dict(color="red", size=10))])

          for k in trange(0, N, 1)]
  #         frames = [go.Frame(data=i) for i in get_markers(zx)]

  )
  fig.update_layout(font_color='#E4D6A7',title_font_color="#E4D6A7",legend_font_color='#E4D6A7', legend_title_font_color="#E4D6A7",title_x=0.5,plot_bgcolor = BG_PIC_COLOR, paper_bgcolor=PAPER_BGCOLOR,xaxis_title="Długość geograficzna", yaxis_title = "Szerokość geograficzna")
  fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='#807875')
  fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='#807875')
  return fig

# Dashboard creation

In [ ]:
external_stylesheets = [dbc.themes.SLATE]
app = JupyterDash(__name__,external_stylesheets=external_stylesheets)
app.title = "Badania przyspieszen"
server = app.server

Funkcja stworzona do wyboru okrażeń i odpowiadających im położeń

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/analiza danych/wizualizacja danych/trening_18_05_2022/trening.csv')
laps = df['LAP ID'].unique()
value = [2,6]
df_based_on_laps = df.loc[(df['LAP ID'] >= value[0]) & (df['LAP ID'] <= value[1])]
xy_array = df_based_on_laps[['LONGITUDE', 'LATITUDE']].values

In [ ]:
sidebar = html.Div([
        html.H3('Dodatkowe wykresy: '),
        html.Hr(),
        html.P(
            'Histogram przyspieszeń kierunkowych pojazdu.', className='lead'
        ),

        dbc.Row([
                 dbc.Col(dcc.Graph(id='graph1', figure = plot_histogram(df_based_on_laps)))
        ]),

        html.P(
            'Wykres polarny przyspieszeń kierunkowych pojazdu.', className='lead'
        ),

        dbc.Row([
                 dbc.Col(dcc.Graph(id='graph12', figure = plot_acceleration_graph(df_based_on_laps)))
        ]), 
            
    ],
    style=SIDEBAR_STYLE,
)

In [ ]:
content = html.Div([
     html.H3('Grand tour. Measurment of the forces exerted on a racing car during a run on the track.'),
     html.Hr(),
     html.P(
            'The dashboard presents graphs visualising the movement of the vehicle on the track, a histogram and polar plot of vehicle acceleration and a plot of vehicle speed versus time.', className='lead'
        ),
     dbc.Row([
                dbc.Label('To select a lap, grab and move the slider'),               
                dcc.RangeSlider(1, 36, 1, value=[2, 6], id = 'laps_slider'),
            ]),  
     dbc.Row(
                [                
                    dbc.Col(dcc.Graph(id='graph2', figure = show_graph_with_moving_dot(df_based_on_laps)))
                ],
                align="center",
          ),
     dbc.Row(
                [                
                    dbc.Col(dcc.Graph(id='graph3', figure = plot_speed_vs_time_graph(df_based_on_laps)))
                ],
                align="center",
          ),

    ],style=CONTENT_STYLE)

app.layout = html.Div([sidebar, content])

100%|██████████| 2278/2278 [00:01<00:00, 1337.74it/s]


In [ ]:
@app.callback(
    Output(component_id='graph1',component_property='figure'), 
    Output(component_id='graph12',component_property='figure'), 
    Output(component_id='graph2',component_property='figure'), 
    Output(component_id='graph3',component_property='figure'),
    Input(component_id='laps_slider',component_property='value')) #slider

def update_output(input_value):
  dfa = dfa = df.loc[(df['LAP ID'] >= input_value[0]) & (df['LAP ID'] <= input_value[1])]
  return plot_histogram(dfa),plot_acceleration_graph(dfa),  show_graph_with_moving_dot(dfa), plot_speed_vs_time_graph(dfa)

In [ ]:
app.run_server(mode='inline', host='localhost', port=1111)

<IPython.core.display.Javascript object>